In [9]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torch import nn
import matplotlib.pyplot as plt

In [10]:
weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
transforms = weights.transforms()

In [11]:
import requests
import zipfile
from pathlib import Path

# Setup path to data folder
image_path = Path("data/") / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it...
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

    # Download pizza, steak, sushi data
    with open(Path("data/") / "pizza_steak_sushi.zip", "wb") as f:
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
        print("Downloading pizza, steak, sushi data...")
        f.write(request.content)

    # Unzip pizza, steak, sushi data
    with zipfile.ZipFile(Path("data/") / "pizza_steak_sushi.zip", "r") as zip_ref:
        print("Unzipping pizza, steak, sushi data...")
        zip_ref.extractall(image_path)

data\pizza_steak_sushi directory exists.


In [12]:
import os
train_dir = image_path / "train"
test_dir = image_path / "test"

import lib.torchLib as lib
#transforms = lib.get_transforms(["resize64", "flip5", "bin", "norm"])
train_loader, test_loader, class_names = lib.create_dataloader_from_dir(train_dir, test_dir, transforms, 32)

In [13]:
model = torchvision.models.efficientnet_b2(weights=weights)
for param in model.parameters():
    param.requires_grad = False
model.classifier = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(1408, len(class_names))
)

In [14]:
from torchinfo import summary
summary(model,
        input_size=(1, 3, 224, 224),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 224, 224]     [1, 3]               --                   Partial
├─Sequential (features)                                      [1, 3, 224, 224]     [1, 1408, 7, 7]      --                   False
│    └─Conv2dNormActivation (0)                              [1, 3, 224, 224]     [1, 32, 112, 112]    --                   False
│    │    └─Conv2d (0)                                       [1, 3, 224, 224]     [1, 32, 112, 112]    (864)                False
│    │    └─BatchNorm2d (1)                                  [1, 32, 112, 112]    [1, 32, 112, 112]    (64)                 False
│    │    └─SiLU (2)                                         [1, 32, 112, 112]    [1, 32, 112, 112]    --                   --
│    └─Sequential (1)                                        [1, 32, 112, 112]    [1, 1

In [15]:
results = lib.train(model, train_loader, test_loader, nn.CrossEntropyLoss(), torch.optim.Adam(params=model.parameters(), lr=0.001), 10)

 10%|█         | 1/10 [00:48<07:19, 48.86s/it]

Epoch: 1 | train_loss: 1.0392 | train_acc: 0.4805 | test_loss: 0.9079 | test_acc: 0.7036


 20%|██        | 2/10 [01:45<07:05, 53.25s/it]

Epoch: 2 | train_loss: 0.9077 | train_acc: 0.6953 | test_loss: 0.7179 | test_acc: 0.8958


 30%|███       | 3/10 [02:47<06:41, 57.30s/it]

...

Epoch: 8 | train_loss: 0.5176 | train_acc: 0.8242 | test_loss: 0.4862 | test_acc: 0.9072


 90%|█████████ | 9/10 [09:02<01:01, 61.41s/it]

Epoch: 9 | train_loss: 0.4193 | train_acc: 0.9297 | test_loss: 0.4930 | test_acc: 0.8769


100%|██████████| 10/10 [10:08<00:00, 60.82s/it]

Epoch: 10 | train_loss: 0.4116 | train_acc: 0.9219 | test_loss: 0.4968 | test_acc: 0.8873


In [16]:
from torchvision import datasets
train_data = datasets.ImageFolder(train_dir, transform=transforms)
test_data = datasets.ImageFolder(test_dir, transform=transforms)

In [17]:
list(test_data)

[(tensor([[[-1.8439, -1.8439, -1.8782,  ..., -1.7240, -1.6727, -1.5699],
           [-1.8610, -1.8610, -1.8782,  ..., -1.6213, -1.6042, -1.5014],
           [-1.9124, -1.9124, -1.9124,  ..., -1.5528, -1.5528, -1.4843],
           ...,
           [-1.8610, -1.8610, -1.8610,  ..., -1.9467, -1.8953, -1.7925],
           [-1.8610, -1.8439, -1.8610,  ..., -1.8953, -1.9467, -1.8782],
           [-1.8439, -1.8439, -1.8439,  ..., -1.7412, -1.8782, -1.8268]],
  
          [[-1.9482, -1.9657, -1.9482,  ..., -1.9657, -1.9132, -1.8081],
           [-1.9832, -1.9657, -1.9657,  ..., -1.8606, -1.8431, -1.7381],
           [-1.9832, -1.9832, -1.9832,  ..., -1.7906, -1.7906, -1.7206],
           ...,
           [-2.0182, -2.0182, -2.0182,  ..., -1.8606, -1.8081, -1.7381],
           [-2.0182, -2.0182, -2.0182,  ..., -1.8081, -1.8606, -1.8256],
           [-2.0007, -2.0007, -2.0007,  ..., -1.6506, -1.7906, -1.7731]],
  
          [[-1.7870, -1.7870, -1.8044,  ..., -1.8044, -1.7870, -1.6824],
           

In [ ]:
lib.plot_wrong_preds(test_data, model, class_names)

In [1]:
lib.save_model(model, "", "effnet2")

NameError: name 'lib' is not defined

In [ ]:
def make_predictions(model, data: list):
    pred_probs = []
    model.eval()
    with torch.inference_mode():
        for sample in data:
            #Prepare sample
            sample = torch.unsqueeze(sample, dim=0).to(device)
            pred_logit = model.forward(sample)
            pred_prob = torch.softmax(pred_logit.squeeze(), dim=0)
            # Get pred_prob off GPU for further calculations
            pred_probs.append(pred_prob.cpu())
    return torch.stack(pred_probs)

plt.figure(figsize=(12, 12))
num=5
import math
import random
nrows = round(math.sqrt(num))
ncols = round(num/nrows)

all_samples = []
all_labels = []
for sample, label in random.sample(list(test_data), k=5):
    all_samples.append(sample)
    all_labels.append(label)

all_pred_probs = make_predictions(model, all_samples)
all_pred_classes = all_pred_probs.argmax(dim=1)

showed = 0
for i, sample in enumerate(all_samples):
    # Find the prediction label (in text form, e.g. "Sandal")
    pred_label = class_names[all_pred_classes[i]]

    # Get the truth label (in text form, e.g. "T-shirt")
    truth_label = class_names[all_labels[i]]

    # Check for equality and change title colour accordingly
    if pred_label != truth_label:
        if showed == 20:
            break
        plt.subplot(nrows, ncols, showed+1)
        plt.imshow(sample.squeeze(), cmap="gray")
        title_text = f"Pred: {pred_label} | Truth: {truth_label}"
        plt.title(title_text, fontsize=10, c="r") # red text if wrong
        showed+=1
    plt.axis=False;

In [3]:
from transformers import pipeline
pipe = pipeline("quesion-generation")

KeyError: "Unknown task quesion-generation, available tasks are ['audio-classification', 'automatic-speech-recognition', 'conversational', 'depth-estimation', 'document-question-answering', 'feature-extraction', 'fill-mask', 'image-classification', 'image-segmentation', 'image-to-text', 'ner', 'object-detection', 'question-answering', 'sentiment-analysis', 'summarization', 'table-question-answering', 'text-classification', 'text-generation', 'text2text-generation', 'token-classification', 'translation', 'visual-question-answering', 'vqa', 'zero-shot-classification', 'zero-shot-image-classification', 'zero-shot-object-detection', 'translation_XX_to_YY']"